In [1]:
from models import SGCRes
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 100
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [3]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\dgl\data\utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [4]:
model = SGCRes(in_feats, n_classes, 2, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

In [6]:
train(model, g, features, labels, train_mask, val_mask, test_mask)

Epoch 01, Loss: 1.2811, Train: 0.4667, Val: 0.5400, Test: 0.5510
Epoch 02, Loss: 1.2265, Train: 0.6500, Val: 0.6300, Test: 0.6240
Epoch 03, Loss: 0.9249, Train: 0.8333, Val: 0.7060, Test: 0.6780
Epoch 04, Loss: 0.6692, Train: 0.8500, Val: 0.6640, Test: 0.6550
Epoch 05, Loss: 0.5981, Train: 0.8667, Val: 0.6500, Test: 0.6440
Epoch 06, Loss: 0.5807, Train: 0.8667, Val: 0.6360, Test: 0.6430
Epoch 07, Loss: 0.4932, Train: 0.8833, Val: 0.6540, Test: 0.6420
Epoch 08, Loss: 0.4289, Train: 0.9000, Val: 0.6820, Test: 0.6720
Epoch 09, Loss: 0.3905, Train: 0.9000, Val: 0.6920, Test: 0.6960
Epoch 10, Loss: 0.3687, Train: 0.9167, Val: 0.7200, Test: 0.7150
Epoch 11, Loss: 0.3410, Train: 0.9333, Val: 0.7520, Test: 0.7230
Epoch 12, Loss: 0.3133, Train: 0.9333, Val: 0.7500, Test: 0.7460
Epoch 13, Loss: 0.2980, Train: 0.9333, Val: 0.7540, Test: 0.7480
Epoch 14, Loss: 0.2862, Train: 0.9333, Val: 0.7580, Test: 0.7520
Epoch 15, Loss: 0.2744, Train: 0.9333, Val: 0.7620, Test: 0.7530
Epoch 16, Loss: 0.2618, T